In [ ]:
import os
import re
import pandas as pd 

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#2_text'}.xlsx")
df = pd.read_excel(xname, index_col=0)
result_list = []

In [ ]:
for i in range(len(df)):
    
    seqnum = df.loc[i, 'seqnum']
    if seqnum in [17843, 16973, 16969, 14646]: # 사분할 파일인 경우 제외
        continue
        
    cbub_code = df.loc[i, 'cbub_code'];title = df.loc[i, "title"];url = df.loc[i, 'url']
    path = df.loc[i, 'path'];text = df.loc[i, 'text'];image = df.loc[i, 'image']
    
    if '원심판결' in text:
        appeal = 1
    else:
        appeal = 0
    
    text = re.sub(r"(-\s?\d\s?-)", ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'_{2,}', '', text)
    
    try:
        info = re.sub(r'\n', '', text[text.index('사 건')+3:text.index('피고인 겸')]) #사건 번호가 없으면 파일 앞 부분 잘린 것.
        info = re.sub(r'\s{2,}', '', info)
    except:
        try:
            info = re.sub(r'\n', '', text[text.index('사 건')+3:text.index('피    고   인')])
            info = re.sub(r'\s{2,}', '', info)
        except:
            try:
                info = re.sub(r'\n', '', text[text.index('사 건')+3:text.index('피 고 인')])
                info = re.sub(r'\s{2,}', '', info)
            except:
                try:
                    info = re.sub(r'\n', '', text[text.index('사 건')+3:text.index('피고 인 겸')])
                    info = re.sub(r'\s{2,}', '', info)
                except:
                    try:
                        info = re.sub(r'\n', '', text[text.index('사 건')+3:text.index('피 고 인 겸, 피부착명령청구자')])
                        info = re.sub(r'\s{2,}', '', info)
                    except:
                        try:
                            info = re.search(r'\b사 건\b(.+)', text).group(1)
                        except:
                            continue
    
    try:
        info = info[:info.index('피고')]
    except:
        pass
    
    try:
        info = info[:info.index('피 고')]
    except:
        pass
    
    try:
        info = info[:info.index('원 고')]
    except:
        pass
    
    info = info.replace('번 호', '').replace('일부', '').replace('인정된', '').replace('죄명', '').replace('', '').replace('예비적', '')
    info = info.replace('인정한', '').replace('춘천', '').replace('제주', '').replace('청주', '').replace('창원', '').replace('변경된', '')
    info = re.sub(r'[가-하]\.', '', info)
    info = re.sub(r'\(.+?\s.+?\)', '', info)
    
    try:
        cid = re.findall(r'\d+\w+\d+', info)[0]
    except:
        continue
    
    case = list(set([i for i in re.findall(r"\w{2,}", info) if i.isalpha()]))
    for i in range(len(case)):
        if case[i][-1]=='등':
            case[i] = case[i][:-1]
        if case[i] == '사가':
            case[i] = '사기'

    undertrial = re.findall(r'\b피 고 인\s(.+)[\n{2,}]+(.+)+', text)
    
    try:
        sentence = re.sub(r'\n{2,}', ' ', text[text.index('주 문')+3:text.index('이 유')])
        if '재판장' in text:
            body = text[text.index('이 유'):text.rindex('재판장')]
        elif '판사' in text:
            body = text[text.index('이 유'):text.rindex('판사')]
    except:
        try:
            sentence = re.sub(r'\n{2,}', ' ', text[text.index('판 결 선 고')+7:text.index('판 결 선 고')+200])
            if '재판장' in text:
                body = text[text.index('판 결 선 고')+200:text.rindex('재판장')]
            elif '판사' in text:
                body = text[text.index('판 결 선 고')+200:text.rindex('판사')]
            else:
                body = text[text.index('판 결 선 고')+200:]
        except:
            continue
                
    body = body.replace('이 유', '').replace('범 죄 사 실', '').replace('증거의 요지', '').replace('법령의 적용', '').replace('양형의 이유', '')
    body = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', body)
    body = re.sub("\n{2,}", "", body)
    body = re.sub(r'(\d+\.\s\d+\.\s\d+\.)(\s\d+:\d+경)?', '', body)
    body = re.sub(r'[\(\[].*?[\)\]]', '', body)
    
    new_dict={
        'seqnum': seqnum,
        'cbub_code': cbub_code,
        'title': title,
        'url': url,
        'path': path,
        'text': text, 
        'image': image,
        'appeal': appeal,
        'cid': cid,
        'case': case,
        'undertrial': undertrial,
        'sentence': sentence,
        'body': body
    }
    
    result_list.append(new_dict)
    
df = pd.DataFrame(result_list)        
save_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#3_columnize'}.xlsx")
excel_writer = pd.ExcelWriter(save_name, engine='xlsxwriter')
df.to_excel(excel_writer)
excel_writer.save()
print(len(df))